# Environement Setup

In [1]:
# @title 1. Setup Progetto (MistakeDetection)
import sys, os
try:
    from google.colab import drive, userdata
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

REPO_NAME = 'MistakeDetection'

# --- CONFIGURAZIONE PATH ---
if IS_COLAB:
    print("☁️ Colab rilevato.")
    if not os.path.exists('/content/drive'):
        drive.mount('/content/drive')

    # DEFINIZIONE GLOBALE PROJECT_DIR (Importante per le celle successive!)
    PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"

    # Fallback se la cartella ha un nome diverso
    if not os.path.exists(PROJECT_DIR):
        if os.path.exists("/content/drive/MyDrive/CaptainCook4D"):
            PROJECT_DIR = "/content/drive/MyDrive/CaptainCook4D"

    print(f"📂 Cartella Progetto su Drive: {PROJECT_DIR}")

    GITHUB_USER = 'MarcoPernoVDP'
    try:
        TOKEN = userdata.get('GITHUB_TOKEN')
        REPO_URL = f'https://{TOKEN}@github.com/{GITHUB_USER}/{REPO_NAME}.git'
    except:
        REPO_URL = f'https://github.com/{GITHUB_USER}/{REPO_NAME}.git'

    ROOT_DIR = f'/content/{REPO_NAME}'

    if not os.path.exists(ROOT_DIR):
        print(f"📥 Clonazione {REPO_NAME}...")
        !git clone {REPO_URL}
    else:
        print(f"🔄 Aggiornamento {REPO_NAME}...")
        %cd {ROOT_DIR}
        !git pull
        %cd /content
else:
    print("Ambiente locale rilevato.")
    ROOT_DIR = os.getcwd()
    while not os.path.exists(os.path.join(ROOT_DIR, '.gitignore')) and ROOT_DIR != os.path.dirname(ROOT_DIR):
        ROOT_DIR = os.path.dirname(ROOT_DIR)
    PROJECT_DIR = ROOT_DIR # In locale coincidono spesso

if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

print(f"✅ ROOT_DIR impostata a: {ROOT_DIR}")

☁️ Colab rilevato.
Mounted at /content/drive
📂 Cartella Progetto su Drive: /content/drive/MyDrive/MistakeDetection
📥 Clonazione MistakeDetection...
Cloning into 'MistakeDetection'...
remote: Enumerating objects: 550, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 550 (delta 7), reused 8 (delta 4), pack-reused 533 (from 1)
Receiving objects: 100% (550/550), 85.58 MiB | 29.92 MiB/s, done.
Resolving deltas: 100% (279/279), done.
✅ ROOT_DIR impostata a: /content/MistakeDetection


In [2]:
# @title 2. Setup ActionFormer
import os
import sys
import shutil
import subprocess

# Usiamo un workspace separato
AF_WORKDIR = "/content/actionformer_workspace"
os.makedirs(AF_WORKDIR, exist_ok=True)
os.chdir(AF_WORKDIR)

REPO_NAME = "multi_step_localization"
AF_REPO_PATH = os.path.join(AF_WORKDIR, REPO_NAME)

# 1. Clone
if not os.path.exists(AF_REPO_PATH):
    print("📥 Clonazione ActionFormer (con --recursive)...")
    try:
        subprocess.run(["git", "clone", "--recursive", "https://github.com/CaptainCook4D/multi_step_localization.git"], check=True)
    except Exception as e:
        print(f"⚠️ Clone recursive fallito, provo standard...")
        subprocess.run(["git", "clone", "https://github.com/CaptainCook4D/multi_step_localization.git"], check=True)

os.chdir(AF_REPO_PATH)

# 2. Fix Path Libs
if os.path.exists(os.path.join(AF_REPO_PATH, "actionformer", "libs", "utils")):
    UTILS_PATH = os.path.join(AF_REPO_PATH, "actionformer", "libs", "utils")
    PATCH_DIR = os.path.join(AF_REPO_PATH, "actionformer")
elif os.path.exists(os.path.join(AF_REPO_PATH, "libs", "utils")):
    UTILS_PATH = os.path.join(AF_REPO_PATH, "libs", "utils")
    PATCH_DIR = AF_REPO_PATH
else:
    # Tentativo update submodule
    print("⚠️ Cartella libs non trovata, provo update submodule...")
    subprocess.run(["git", "submodule", "update", "--init", "--recursive"], check=True)
    if os.path.exists(os.path.join(AF_REPO_PATH, "libs", "utils")):
        UTILS_PATH = os.path.join(AF_REPO_PATH, "libs", "utils")
        PATCH_DIR = AF_REPO_PATH
    else:
        raise FileNotFoundError("CRITICO: Impossibile trovare libs/utils.")

print(f"✅ Cartella Utils: {UTILS_PATH}")

# 3. Installazione & Patch
print("📦 Installazione dipendenze...")
subprocess.run([sys.executable, "-m", "pip", "install", "pyyaml", "scipy"], check=True)

print("🩹 Patch NumPy 2.0...")
with open(os.path.join(PATCH_DIR, "numpy_patch.py"), "w") as f:
    f.write("import numpy as np\n")
    f.write("try:\n  if not hasattr(np, 'float'): np.float = np.float64\nexcept: pass\n")
    f.write("try:\n  if not hasattr(np, 'int'): np.int = np.int_\nexcept: pass\n")

# Inietta patch in eval.py
eval_path = os.path.join(AF_REPO_PATH, "eval.py")
if os.path.exists(eval_path):
    with open(eval_path, "r") as f: content = f.read()
    if "import numpy_patch" not in content:
        with open(eval_path, "w") as f:
            f.write("import sys\nsys.path.append('actionformer')\nimport numpy_patch\n" + content)

# 4. Compilazione
print("⚙️ Compilazione CUDA...")
os.chdir(UTILS_PATH)
subprocess.run([sys.executable, "setup.py", "install"], check=True)

os.chdir(AF_REPO_PATH)
print("\n✅ Ambiente ActionFormer pronto.")

📥 Clonazione ActionFormer (con --recursive)...
✅ Cartella Utils: /content/actionformer_workspace/multi_step_localization/actionformer/libs/utils
📦 Installazione dipendenze...
🩹 Patch NumPy 2.0...
⚙️ Compilazione CUDA...

✅ Ambiente ActionFormer pronto.


In [39]:
# @title 3. Estrazione Feature Omnivore
import zipfile
import shutil
import os
from tqdm import tqdm

# --- CONFIGURAZIONE VARIABILI (Self-Contained) ---
if 'PROJECT_DIR' not in locals():
    # Tenta di indovinare il path
    if os.path.exists("/content/drive/MyDrive/MistakeDetection"):
        PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"
    elif os.path.exists("/content/drive/MyDrive/CaptainCook4D"):
        PROJECT_DIR = "/content/drive/MyDrive/CaptainCook4D"
    else:
        # Fallback locale se non trova nulla (o se sei in locale)
        PROJECT_DIR = os.getcwd()

if 'ROOT_DIR' not in locals():
    # Tenta di trovare il repo clonato
    possible_roots = [
        os.path.join(PROJECT_DIR, "MistakeDetection"),
        "/content/MistakeDetection",
        PROJECT_DIR
    ]
    for r in possible_roots:
        if os.path.exists(os.path.join(r, ".git")):
            ROOT_DIR = r
            break
    if 'ROOT_DIR' not in locals(): ROOT_DIR = PROJECT_DIR

# --- RICERCA ZIP ---
POSSIBLE_PATHS = [
    os.path.join(PROJECT_DIR, "_file", "omnivore.zip"),
    os.path.join(PROJECT_DIR, "data", "omnivore.zip"),
    os.path.join(PROJECT_DIR, "omnivore.zip"),
    # Path specifici colab
    "/content/drive/MyDrive/MistakeDetection/omnivore.zip",
    "/content/drive/MyDrive/MistakeDetection/data/omnivore.zip"
]

ZIP_PATH = None
for p in POSSIBLE_PATHS:
    if os.path.exists(p):
        ZIP_PATH = p
        break

LOCAL_FEAT_DIR = "/content/temp_omnivore_features"

if ZIP_PATH is None:
    print(f"❌ ERRORE: Non trovo 'omnivore.zip'.")
    print(f"   Ho cercato in: {POSSIBLE_PATHS}")
else:
    print(f"📂 Trovato Zip: {ZIP_PATH}")
    print(f"⏳ Estrazione in: {LOCAL_FEAT_DIR}...")

    if os.path.exists(LOCAL_FEAT_DIR):
        try:
            shutil.rmtree(LOCAL_FEAT_DIR)
        except: pass # Ignora errori permessi
    os.makedirs(LOCAL_FEAT_DIR, exist_ok=True)

    with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
        zip_ref.extractall(LOCAL_FEAT_DIR)

    print(f"✅ Estrazione completata.")

📂 Trovato Zip: /content/drive/MyDrive/MistakeDetection/omnivore.zip
⏳ Estrazione in: /content/temp_omnivore_features...
✅ Estrazione completata.


# Features Extraction

In [ ]:
# @title 3.5 Riparazione Generazione JSON (Debug & Fallback)
import os
import glob
import subprocess
import json
import sys
import shutil

# --- CONFIGURAZIONE ---
AF_WORKDIR = "/content/actionformer_workspace"
if os.path.exists(os.path.join(AF_WORKDIR, "multi_step_localization")):
    AF_REPO_PATH = os.path.join(AF_WORKDIR, "multi_step_localization")
else:
    AF_REPO_PATH = AF_WORKDIR

if 'PROJECT_DIR' not in locals():
    if os.path.exists("/content/drive/MyDrive/MistakeDetection"):
        PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"
    else:
        PROJECT_DIR = "/content/drive/MyDrive/CaptainCook4D"

USER_ANNOTATION_DIR = os.path.join(PROJECT_DIR, "annotation_json")
TARGET_JSON = os.path.join(AF_WORKDIR, "actionformer_split.json")
TARGET_JSON_REC = os.path.join(AF_WORKDIR, "actionformer_split_recordings.json")

print(f"📂 Cartella Annotazioni Utente: {USER_ANNOTATION_DIR}")

# 1. VERIFICA FILE SORGENTI
if not os.path.exists(USER_ANNOTATION_DIR):
    print("❌ ERRORE: La cartella annotation_json non esiste!")
    print(f"   Crea la cartella: {USER_ANNOTATION_DIR} e mettici dentro i file .json dei video.")
    raise FileNotFoundError("Cartella annotation_json mancante")

json_files = glob.glob(os.path.join(USER_ANNOTATION_DIR, "*.json"))
print(f"   Trovati {len(json_files)} file .json sorgenti.")

if len(json_files) == 0:
    print("⚠️ ATTENZIONE: La cartella annotation_json è VUOTA.")
    print("   Senza file .json input, non possiamo creare il dataset ActionFormer.")

# 2. TENTATIVO 1: USARE LO SCRIPT DEL REPO (Con Debug)
converter_script = os.path.join(AF_REPO_PATH, "convert_to_action_former_json.py")
if os.path.exists(converter_script):
    print(f"🚀 Avvio script conversione ufficiale: {os.path.basename(converter_script)}")

    cmd = [
        "python", converter_script,
        "--annotation_folder", USER_ANNOTATION_DIR,
        "--output_file", TARGET_JSON
    ]

    # Eseguiamo catturando l'output per vedere l'errore
    result = subprocess.run(cmd, capture_output=True, text=True, cwd=AF_REPO_PATH)

    if result.returncode == 0 and os.path.exists(TARGET_JSON):
        print("✅ Conversione ufficiale RIUSCITA!")
    else:
        print("❌ Conversione ufficiale FALLITA.")
        print("--- ERRORE SCRIPT ---")
        print(result.stderr)
        print("---------------------")
        print("⚠️ Procedo con Generazione MANUALE di Emergenza (Fallback)...")

# 3. TENTATIVO 2: GENERATORE MANUALE (Fallback)
# Se lo script sopra fallisce, creiamo noi un JSON valido per ActionFormer
if not os.path.exists(TARGET_JSON):
    print("🛠️ Avvio Generatore Manuale (Python)...")

    database = {}

    # Se abbiamo file json reali, proviamo a leggerli
    if json_files:
        for jf in json_files:
            vid_id = os.path.basename(jf).replace(".json", "")
            try:
                with open(jf, 'r') as f:
                    data = json.load(f)

                # Cerca di capire la struttura (CaptainCook ha varie versioni)
                # Struttura attesa: Lista di step o dizionario
                annotations = []

                # Caso A: Lista diretta di step
                if isinstance(data, list):
                    for item in data:
                        if 'start_time' in item and 'end_time' in item:
                             annotations.append({
                                 "segment": [float(item['start_time']), float(item['end_time'])],
                                 "label": item.get('label', 'unknown_step')
                             })

                # Caso B: Dizionario (es. 'segments': [...])
                elif isinstance(data, dict):
                     # Logica da adattare se necessario
                     pass

                # Se non riusciamo a leggere, creiamo un placeholder per far girare il modello
                if not annotations:
                    # Placeholder: ActionFormer troverà da solo i segmenti
                    # Mettiamo un segmento finto che copre tutto il video (ipotesi)
                    annotations.append({"segment": [0, 1000], "label": "test"})

                database[vid_id] = {
                    "subset": "validation", # Fondamentale per eval.py
                    "annotations": annotations
                }
            except Exception as e:
                print(f"   Errore lettura {vid_id}: {e}")

    # Se non c'erano file o lettura fallita, usiamo le feature presenti
    if not database:
        print("⚠️ Lettura annotazioni fallita. Genero DB basato sui file Feature (.npz)...")
        # Leggiamo la cartella feature per sapere quali video abbiamo
        local_feat_dir = "/content/temp_omnivore_features"
        if os.path.exists(local_feat_dir):
            feat_files = glob.glob(os.path.join(local_feat_dir, "*.npz"))
            for ff in feat_files:
                vid_id = os.path.basename(ff).replace(".npz", "")
                # Creiamo una entry dummy valida
                database[vid_id] = {
                    "subset": "validation",
                    "annotations": [{"segment": [0.0, 1.0], "label": "dummy"}]
                }

    # Salva il JSON finale
    final_data = {"database": database}
    with open(TARGET_JSON, 'w') as f:
        json.dump(final_data, f)
    print(f"✅ JSON generato manualmente: {len(database)} video inseriti.")

# 4. DUPLICAZIONE PER COMPATIBILITÀ (Il trucco _recordings)
if os.path.exists(TARGET_JSON):
    shutil.copy2(TARGET_JSON, TARGET_JSON_REC)
    print(f"✅ Creato duplicato necessario: {os.path.basename(TARGET_JSON_REC)}")
    print("🎉 ORA PUOI ESEGUIRE LA CELLA 4!")
else:
    print("❌ DISASTRO: Impossibile creare il file JSON in nessun modo.")

In [7]:
# @title 3.6 Fix JSON Taxonomy (Risolve KeyError: 'label_id')
import json
import os
import shutil

# CONFIGURAZIONE
AF_WORKDIR = "/content/actionformer_workspace"
JSON_PATH = os.path.join(AF_WORKDIR, "actionformer_split.json")
JSON_REC_PATH = os.path.join(AF_WORKDIR, "actionformer_split_recordings.json")

print(f"🔧 Analisi file: {JSON_PATH}")

if not os.path.exists(JSON_PATH):
    # Se manca il base, proviamo a prendere il recordings se esiste
    if os.path.exists(JSON_REC_PATH):
        shutil.copy2(JSON_REC_PATH, JSON_PATH)
        print("   Recuperato da _recordings.json")
    else:
        raise FileNotFoundError("Nessun file JSON trovato da riparare!")

# 1. CARICAMENTO
with open(JSON_PATH, 'r') as f:
    data = json.load(f)

db = data.get('database', {})
if not db:
    # Se il json è piatto, prova a ristrutturarlo
    print("⚠️ Struttura 'database' mancante. Tento ristrutturazione...")
    db = data
    data = {'database': db}

# 2. RACCOLTA ETICHETTE
unique_labels = set()
for vid_id, vid_data in db.items():
    annotations = vid_data.get('annotations', [])
    for ann in annotations:
        if 'label' in ann:
            unique_labels.add(ann['label'])

sorted_labels = sorted(list(unique_labels))
print(f"✅ Trovate {len(sorted_labels)} classi uniche.")
print(f"   Esempio: {sorted_labels[:5]}...")

# 3. CREAZIONE TASSONOMIA (Il pezzo mancante!)
# Thumos si aspetta una lista di dizionari con 'nodeName'/'label' e 'nodeId'/'label_id'
# Analizzando l'errore: label_dict[act['label']] = act['label_id']
# Quindi serve una lista che abbia 'label' e 'label_id'.

taxonomy = []
for idx, label in enumerate(sorted_labels):
    entry = {
        "id": idx,
        "label": label,      # Fondamentale per il fix
        "label_id": idx,     # Fondamentale per il fix
        "nodeId": idx,       # Extra sicurezza
        "nodeName": label,   # Extra sicurezza
        "parentId": None
    }
    taxonomy.append(entry)

# Inseriamo la tassonomia nel JSON
data['taxonomy'] = taxonomy
print(f"✅ Sezione 'taxonomy' aggiunta con {len(taxonomy)} voci.")

# 4. SALVATAGGIO
with open(JSON_PATH, 'w') as f:
    json.dump(data, f, indent=4)

# Aggiorniamo anche la copia _recordings per sicurezza
shutil.copy2(JSON_PATH, JSON_REC_PATH)

print(f"💾 File salvati e riparati:\n   -> {os.path.basename(JSON_PATH)}\n   -> {os.path.basename(JSON_REC_PATH)}")
print("🚀 ORA ESEGUI LA CELLA 4 (Dovrebbe funzionare!)")

🔧 Analisi file: /content/actionformer_workspace/actionformer_split.json
✅ Trovate 1 classi uniche.
   Esempio: ['test']...
✅ Sezione 'taxonomy' aggiunta con 1 voci.
💾 File salvati e riparati:
   -> actionformer_split.json
   -> actionformer_split_recordings.json
🚀 ORA ESEGUI LA CELLA 4 (Dovrebbe funzionare!)


In [44]:
# @title 4. Inferenza ActionFormer (Auto-Detect, Fix Totali & Verbose Log)
import os
import glob
import subprocess
import yaml
import sys
import shutil
import json
import torch
import numpy as np
import time

# Forza output immediato senza buffering
os.environ['PYTHONUNBUFFERED'] = '1'

print("🚀 Inizializzazione ActionFormer (VERBOSE MODE)...", flush=True)

# --- 1. CONFIGURAZIONE AMBIENTE ---
AF_WORKDIR = "/content/actionformer_workspace"
AF_REPO_PATH = os.path.join(AF_WORKDIR, "multi_step_localization")
if not os.path.exists(AF_REPO_PATH): AF_REPO_PATH = AF_WORKDIR

if AF_REPO_PATH not in sys.path: sys.path.append(AF_REPO_PATH)

if 'PROJECT_DIR' not in locals():
    if os.path.exists("/content/drive/MyDrive/MistakeDetection"):
        PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"
    else:
        PROJECT_DIR = "/content/drive/MyDrive/CaptainCook4D"

# --- 2. CACCIA AL TESORO (TROVA I FILE .NPZ) ---
print("🔍 [System] Ricerca file feature...", flush=True)
search_paths = [
    "/content/temp_omnivore_features/omnivore", # Path specificato da te
    "/content/temp_omnivore_features",
    os.path.join(PROJECT_DIR, "features", "omnivore_video"),
    os.path.join(PROJECT_DIR, "features"),
    os.path.join(PROJECT_DIR, "omnivore_features"),
    "/content/omnivore_features",
    "/content/features"
]

FOUND_FEAT_DIR = None
num_files = 0

for path in search_paths:
    if os.path.exists(path):
        files = glob.glob(os.path.join(path, "*.npz"))
        if len(files) > 0: # Anche pochi file vanno bene
            FOUND_FEAT_DIR = path
            num_files = len(files)
            print(f"✅ Trovati {num_files} file in: {path}", flush=True)
            break

if not FOUND_FEAT_DIR:
    print("❌ ERRORE: Non trovo i file .npz!", flush=True)
    # Creiamo dummy per test se non trova nulla
    FOUND_FEAT_DIR = "/content/temp_dummy"
    os.makedirs(FOUND_FEAT_DIR, exist_ok=True)
    print("⚠️ Uso cartella dummy per evitare crash.", flush=True)

LOCAL_FEAT_DIR = FOUND_FEAT_DIR
MY_CONFIG_PATH = os.path.join(AF_REPO_PATH, "configs", "forced_config.yaml")
DATASETS_LIB_PATH = os.path.join(AF_REPO_PATH, "actionformer", "libs", "datasets")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️ Hardware: {DEVICE.upper()}", flush=True)


# --- 3. GENERAZIONE JSON SU FILE REALI ---
print(f"🔍 [System] Generazione JSON per {num_files} video...", flush=True)
json_base = os.path.join(AF_WORKDIR, "actionformer_split.json")

full_db = {}
feature_files = glob.glob(os.path.join(LOCAL_FEAT_DIR, "*.npz"))

for f in feature_files:
    vid_name = os.path.basename(f).replace(".npz", "")
    full_db[vid_name] = {
        "subset": "validation",
        "annotations": [{"label": "test", "segment": [0, 1]}]
    }

# Se vuoto aggiungi un dummy
if not full_db:
    full_db["dummy"] = {"subset": "validation", "annotations": [{"label": "test", "segment": [0, 1]}]}

with open(json_base, 'w') as f:
    json.dump({
        "database": full_db,
        "taxonomy": [{"id":0,"label":"test","label_id":0,"nodeName":"test"}],
        "version": "1.0"
    }, f)
shutil.copy2(json_base, json_base.replace(".json","")+"_recordings.json")
print(f"✅ [System] JSON pronto.", flush=True)


# --- 4. RISCRITTURA LIBRERIE (Fix NPZ, Dim, Duration) ---
print("🛠️ [System] Patching librerie ActionFormer...", flush=True)
pycache_dir = os.path.join(DATASETS_LIB_PATH, "__pycache__")
if os.path.exists(pycache_dir): shutil.rmtree(pycache_dir)

# A. __INIT__.PY
with open(os.path.join(DATASETS_LIB_PATH, "__init__.py"), 'w') as f:
    f.write("from .datasets import make_dataset, make_data_loader\n")

# B. DATASETS.PY
datasets_code = r"""
import torch
import os

_DATASET_REGISTRY = {}
def register_dataset(name):
    def decorator(cls):
        _DATASET_REGISTRY[name] = cls
        return cls
    return decorator

def make_dataset(name, is_training, split, **kwargs):
    if name == 'thumos':
        from .thumos14 import THUMOS14Dataset
        return THUMOS14Dataset(is_training, split, **kwargs)
    if name in _DATASET_REGISTRY:
        return _DATASET_REGISTRY[name](is_training, split, **kwargs)
    raise KeyError(f"Dataset sconosciuto: {name}")

def make_data_loader(dataset, is_training, generator, batch_size, num_workers):
    persistent = True if num_workers > 0 else False
    loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, num_workers=num_workers,
        shuffle=(True if is_training else False), collate_fn=None,
        pin_memory=False,
        drop_last=(True if is_training else False),
        persistent_workers=persistent
    )
    return loader
"""
with open(os.path.join(DATASETS_LIB_PATH, "datasets.py"), 'w') as f: f.write(datasets_code)

# C. THUMOS14.PY (ROBUST NPZ LOADER)
thumos_code = r"""
import os
import json
import numpy as np
import torch
from torch.utils.data import Dataset
from .datasets import register_dataset

@register_dataset("thumos")
class THUMOS14Dataset(Dataset):
    def __init__(self, is_training, split, feat_folder, json_file, feat_stride, num_frames,
                 default_fps, downsample_rate, max_seq_len, trunc_thresh, crop_ratio,
                 input_dim, num_classes, file_prefix, file_ext, force_upsampling):
        self.split = split
        self.feat_folder = feat_folder
        self.json_file = json_file
        self.feat_stride = feat_stride
        self.num_frames = num_frames
        self.default_fps = default_fps
        self.downsample_rate = downsample_rate
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.file_prefix = file_prefix
        self.file_ext = file_ext
        dict_db, label_dict = self._load_json_db(self.json_file)
        self.data_list = [val for key, val in dict_db.items()]
        self.label_dict = label_dict

    def get_attributes(self): return self.data_list, self.label_dict, self.num_classes
    def _load_json_db(self, json_file):
        with open(json_file, 'r') as fid: json_data = json.load(fid)
        label_dict = {'test': 0}
        if 'taxonomy' in json_data:
            for act in json_data['taxonomy']:
                label_dict[act.get('label', 'unknown')] = act.get('id', 0)
        dict_db = json_data['database']
        for vid in dict_db:
            if 'id' not in dict_db[vid]: dict_db[vid]['id'] = vid
        return dict_db, label_dict
    def __len__(self): return len(self.data_list)

    def __getitem__(self, idx):
        item = self.data_list[idx]
        feat_file = os.path.join(self.feat_folder, self.file_prefix + item['id'] + self.file_ext)

        # --- ROBUST LOADING (FIXED) ---
        try:
            if not os.path.exists(feat_file):
                feats = np.zeros((self.input_dim, 100), dtype=np.float32)
            else:
                loaded = np.load(feat_file)
                if isinstance(loaded, np.lib.npyio.NpzFile):
                    keys = loaded.files
                    if 'feats' in keys: feats = loaded['feats']
                    elif 'arr_0' in keys: feats = loaded['arr_0']
                    else: feats = loaded[keys[0]]
                else:
                    feats = loaded

            feats = feats.astype(np.float32)

            # Fix Dimensions: Ensure (Channels, Time) -> (1024, N)
            if feats.ndim == 2:
                if feats.shape[1] == self.input_dim: # Se è (N, 1024) -> Trasponi
                    feats = feats.transpose()
        except Exception as e:
            print(f"Error reading {feat_file}: {e}")
            feats = np.zeros((self.input_dim, 100), dtype=np.float32)

        if self.downsample_rate > 1: feats = feats[:, ::self.downsample_rate]
        feat_stride = self.feat_stride * self.downsample_rate

        num_feat_frames = feats.shape[1]
        duration = (num_feat_frames * feat_stride) / self.default_fps

        feats = torch.from_numpy(np.ascontiguousarray(feats))

        return {
            'video_id': item['id'], 'feats': feats,
            'segments': torch.zeros((0, 2), dtype=torch.float32),
            'labels': torch.zeros((0), dtype=torch.int64),
            'fps': self.default_fps, 'feat_stride': feat_stride,
            'feat_num_frames': num_feat_frames, 'duration': duration
        }
"""
with open(os.path.join(DATASETS_LIB_PATH, "thumos14.py"), 'w') as f: f.write(thumos_code)


# --- 5. CONFIGURAZIONE ---
cands = glob.glob(os.path.join(PROJECT_DIR, "**", "*omnivore*.pth*"), recursive=True)
if not cands: raise FileNotFoundError("❌ Modello non trovato!")
MODEL_CKPT = sorted(cands)[-1]

config_data = {
    'dataset_name': 'thumos',
    'model_name': 'LocPointTransformer',
    'output_folder': './ckpt/',
    'devices': [DEVICE],
    'dataset': {
        'json_file': json_base, 'feat_folder': LOCAL_FEAT_DIR, 'file_prefix': '', 'file_ext': '.npz',
        'input_dim': 1024, 'feat_stride': 16, 'num_classes': 24, 'default_fps': 30, 'num_frames': 32,
        'downsample_rate': 1, 'max_seq_len': 2304, 'trunc_thresh': 0.5, 'crop_ratio': None, 'force_upsampling': False,
    },
    'eval': {'batch_size': 1, 'nms_score_thres': 0.1},
    'loader': {'batch_size': 1, 'num_workers': 0},
    'model': {
        'backbone_type': 'convTransformer', 'fpn_type': 'identity', 'backbone_arch': [2, 2, 5],
        'scale_factor': 2, 'input_dim': 1024, 'max_seq_len': 2304, 'n_head': 4,
        'embd_kernel_size': 3, 'embd_with_ln': True, 'fpn_with_ln': True, 'fpn_start_level': 0,
        'head_num_layers': 3, 'head_kernel_size': 3, 'head_with_ln': True, 'use_rel_pe': False,
        'num_classes': 24,
        'regression_range': [[0, 4], [4, 8], [8, 16], [16, 32], [32, 64], [64, 10000]],
        'embd_dim': 512, 'fpn_dim': 512, 'head_dim': 512,
        'use_abs_pe': False, 'max_buffer_len_factor': 6.0, 'n_mha_win_size': 19,
        'train_cfg': {
            'center_sample': 'radius', 'center_sample_radius': 1.5, 'loss_weight': 1.0,
            'cls_prior_prob': 0.01, 'init_loss_norm': 2000, 'clip_grad_l2norm': -1,
            'label_smoothing': 0.0, 'dropout': 0.1, 'droppath': 0.1, 'head_empty_cls': []
        },
        'test_cfg': {
            'pre_nms_thresh': 0.001, 'pre_nms_topk': 5000, 'iou_threshold': 0.1, 'min_score': 0.01,
            'max_seg_num': 1000, 'nms_method': 'soft', 'nms_sigma': 0.5, 'voting_thresh': 0.75,
            'multiclass_nms': True, 'duration_thresh': 0.05
        }
    },
    'train': {'head_dim': 512}
}
with open(MY_CONFIG_PATH, 'w') as f: yaml.dump(config_data, f)


# --- 6. EVAL SCRIPT (Con LOG Dettagliati) ---
eval_standalone = r"""
import os
import torch
import torch.nn as nn
import argparse
import numpy as np
import yaml
import pickle
import json
import sys
import time
from tqdm import tqdm

def log(msg): print(f"[DEBUG] {msg}", flush=True)
def load_config(path):
    with open(path, 'r') as f: return yaml.safe_load(f)

from actionformer.libs.modeling import make_meta_arch
from actionformer.libs.datasets import make_dataset, make_data_loader

def main():
    log("1. Setup Iniziale...")
    config_path = r'""" + MY_CONFIG_PATH + r"""'
    cfg = load_config(config_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    log(f"   -> Device: {device}")

    # Params Clean
    toxic = ['backbone', 'division_type', 'videos_type']
    for k in toxic:
        if k in cfg['dataset']: del cfg['dataset'][k]

    log("2. Dataset & Loader...")
    val_dataset = make_dataset(cfg['dataset_name'], False, ['validation'], **cfg['dataset'])
    val_loader = make_data_loader(val_dataset, False, None, **cfg['loader'])
    log(f"   -> Video trovati: {len(val_dataset)}")

    log("3. Modello...")
    model = make_meta_arch(cfg['model_name'], **cfg['model'])
    model = model.to(device)

    ckpt_path = r'""" + MODEL_CKPT + r"""'
    log(f"4. Caricamento Pesi: {os.path.basename(ckpt_path)} (Attendere...)")
    try:
        checkpoint = torch.load(ckpt_path, map_location=device)
        state_dict = checkpoint.get('state_dict_ema', checkpoint.get('model', checkpoint.get('state_dict', checkpoint)))
        model.load_state_dict(state_dict, strict=False)
        log("   -> Pesi caricati correttamente.")
    except Exception as e:
        log(f"   ❌ ERRORE Pesi: {e}")
        return

    model.eval()
    results = {'video_ids': [], 'segment_intervals': [], 'scores': [], 'labels': []}

    log("5. Avvio Inferenza...")
    with torch.no_grad():
        for i, batch in tqdm(enumerate(val_loader), total=len(val_loader), file=sys.stdout):
            try:
                # Prepare Input
                model_inputs = []
                for k in range(len(batch['video_id'])):
                    input_item = {
                        'feats': batch['feats'][k].to(device),
                        'feat_num_frames': batch['feat_num_frames'][k].to(device),
                        'video_id': batch['video_id'][k],
                        'fps': batch['fps'][k].item(),
                        'feat_stride': batch['feat_stride'][k].item(),
                        'duration': batch['duration'][k].item()
                    }
                    model_inputs.append(input_item)

                # Forward Pass
                output = model(model_inputs)

                # Collect
                for k in range(len(output)):
                    results['video_ids'].append(batch['video_id'][k])
                    results['segment_intervals'].append(output[k]['segments'].cpu().numpy())
                    results['scores'].append(output[k]['scores'].cpu().numpy())
                    results['labels'].append(output[k]['labels'].cpu().numpy())
            except Exception as e:
                log(f"⚠️ Errore nel video {batch.get('video_id', '?')}: {e}")
                continue

    out_dir = cfg['output_folder']
    os.makedirs(out_dir, exist_ok=True)

    log("6. Salvataggio Risultati JSON...")
    json_out = {'results': {}}
    for i, vid in enumerate(results['video_ids']):
        segs = results['segment_intervals'][i]
        scrs = results['scores'][i]
        lbls = results['labels'][i]
        res_list = []
        for j in range(len(segs)):
            res_list.append({
                'label': int(lbls[j]), 'score': float(scrs[j]),
                'segment': [float(segs[j][0]), float(segs[j][1])]
            })
        json_out['results'][vid] = res_list

    final_path = os.path.join(out_dir, "results.json")
    with open(final_path, 'w') as f: json.dump(json_out, f)
    log(f"✅ COMPLETATO! Risultati salvati in: {final_path}")

if __name__ == '__main__':
    main()
"""
with open(os.path.join(AF_REPO_PATH, "eval_standalone.py"), 'w') as f: f.write(eval_standalone)

# --- RUN ---
print("🏁 START PROCESS...", flush=True)
os.chdir(AF_REPO_PATH)
cmd = ["python", "eval_standalone.py"]

# Popen per vedere output live
process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1, universal_newlines=True)
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None: break
    if output: print(output.strip(), flush=True)

if process.poll() == 0:
    print("\n🎉 SUCCESSO! Ora vai alla Cella 5.")
else:
    print(f"\n❌ ERRORE (Code {process.poll()}).")

🚀 Inizializzazione ActionFormer (VERBOSE MODE)...
🔍 [System] Ricerca file feature...
✅ Trovati 384 file in: /content/temp_omnivore_features/omnivore
🖥️ Hardware: CPU
🔍 [System] Generazione JSON per 384 video...
✅ [System] JSON pronto.
🛠️ [System] Patching librerie ActionFormer...
🏁 START PROCESS...
[DEBUG] 1. Setup Iniziale...
[DEBUG]    -> Device: cpu
[DEBUG] 2. Dataset & Loader...
[DEBUG]    -> Video trovati: 384
[DEBUG] 3. Modello...
[DEBUG] 4. Caricamento Pesi: ego4d_omnivore.pth.tar (Attendere...)
[DEBUG]    -> Pesi caricati correttamente.
[DEBUG] 5. Avvio Inferenza...

1%|          | 3/384 [00:07<15:13,  2.40s/it]


KeyboardInterrupt: 

# Pooling, Zipping, Upload

In [ ]:
# @title 5. Pooling e Export Finale
import numpy as np
import json
import glob
import shutil
from tqdm import tqdm

if 'PROJECT_DIR' not in locals(): PROJECT_DIR = "/content/drive/MyDrive/MistakeDetection"

RESULTS_JSON = os.path.join(AF_WORKDIR, "output_results", "results.json")
TEMP_EMB_DIR = os.path.join(AF_WORKDIR, "step_embeddings_temp")
if os.path.exists(TEMP_EMB_DIR): shutil.rmtree(TEMP_EMB_DIR)
os.makedirs(TEMP_EMB_DIR, exist_ok=True)

FINAL_DATA_DIR = os.path.join(PROJECT_DIR, "data")
if not os.path.exists(FINAL_DATA_DIR): os.makedirs(FINAL_DATA_DIR, exist_ok=True)
ZIP_OUTPUT_NAME = "step_embeddings"

def load_npz(path):
    try:
        with np.load(path) as data:
            if 'features' in data: return data['features']
            if 'arr_0' in data: return data['arr_0']
            return data[list(data.keys())[0]]
    except: return None

if os.path.exists(RESULTS_JSON):
    print("🚀 Inizio elaborazione embedding...")

    with open(RESULTS_JSON, 'r') as f:
        data = json.load(f)
        preds = data.get('results', data)

    # Mappa file locali delle feature Omnivore
    file_map = {os.path.basename(f).split('.')[0]: f for f in glob.glob(os.path.join(LOCAL_FEAT_DIR, "*.npz"))}

    count = 0
    fps, stride = 30.0, 16.0
    feat_rate = fps / stride

    print(f"   Analisi di {len(preds)} video...")

    for video_id, segments in tqdm(preds.items()):
        feat_path = file_map.get(video_id)
        if not feat_path:
            # Try fuzzy match
            candidates = [f for k, f in file_map.items() if video_id in k]
            if candidates: feat_path = candidates[0]

        if not feat_path: continue

        full_feats = load_npz(feat_path)
        if full_feats is None: continue

        step_embeddings = []
        step_metadata = []

        for seg in segments:
            t_start, t_end = seg['segment']
            # Filtriamo segmenti troppo brevi o con score bassissimo
            if seg['score'] < 0.05: continue # Abbassato leggermente per essere inclusivi

            s_idx, e_idx = int(t_start * feat_rate), int(t_end * feat_rate)
            s_idx, e_idx = max(0, s_idx), min(len(full_feats), e_idx)
            if e_idx <= s_idx: e_idx = s_idx + 1

            if s_idx < len(full_feats):
                # MEDIA delle feature nel segmento (Pooling)
                pooled = np.mean(full_feats[s_idx:e_idx], axis=0)
                step_embeddings.append(pooled)
                step_metadata.append(seg)

        if step_embeddings:
            np.savez_compressed(
                os.path.join(TEMP_EMB_DIR, f"{video_id}_steps.npz"),
                embeddings=np.array(step_embeddings),
                metadata=step_metadata
            )
            count += 1

    print(f"\n✅ Generati {count} file .npz in {TEMP_EMB_DIR}")

    # Zipping e Copy su Drive
    print(f"📦 Creazione archivio ZIP...")
    zip_path = shutil.make_archive(
        base_name=os.path.join(AF_WORKDIR, ZIP_OUTPUT_NAME),
        format='zip',
        root_dir=TEMP_EMB_DIR
    )

    print(f"☁️ Upload su Drive: {FINAL_DATA_DIR}...")
    try:
        shutil.copy2(zip_path, FINAL_DATA_DIR)
        print(f"✅ SUCCESSO! File salvato in: {os.path.join(FINAL_DATA_DIR, ZIP_OUTPUT_NAME + '.zip')}")
    except Exception as e:
        print(f"❌ Errore durante l'upload: {e}")

else:
    print("❌ Nessun risultato inferenza trovato. Verifica che la Cella 4 abbia finito senza errori.")